# Day5 — Vib Robust-Z Anomaly (Visualization-first) + Event Context
목표:
- processed 데이터에서 Vib_rms의 robust z-score(|z|) 기반 이상치(초 단위) 탐지
- 2-panel (Vib context + |z| score) 시각화 생성
- 멀티센서 개요(RPM/Torque/Vib + |z|) 시각화 생성
- 이벤트 컨텍스트(±window) 그래프를 저장하고, 노트북에서는 지정한 5개 이벤트만 확인

이번 Day5에서는 **모델링보다 시각화 중심**으로 이상치 후보 구간을 빠르게 확인하는 데 초점을 둔다.


## 입력/출력
- 입력(processed): `data/processed/processed_v1_run_abs_features.csv`
- 출력(그림): `figures/Day5/`
  - `day5_vib_robust_score_panel.png`
  - `day5_multisensor_overview.png`
  - `figures/Day5/events/` 아래 이벤트별 컨텍스트 그림
- 실행 스크립트: `src/day5_anomaly_detection.py`


## 1) 데이터 로드 & 기본 확인 (컬럼/스케일 sanity check)


In [ ]:
from pathlib import Path
import pandas as pd

PROCESSED = Path("../../data/processed/processed_v1_run_abs_features.csv")
TABLES_DIR = Path("../../tables/Day5")
FIG_DIR = Path("../../figures/Day5")
EVENT_DIR = FIG_DIR / "events"

PROCESSED.exists(), TABLES_DIR.exists(), FIG_DIR.exists(), EVENT_DIR.exists()

In [ ]:
df = pd.read_csv(PROCESSED)
df.shape, df.columns[:30]

In [ ]:
# 핵심 컬럼 존재 여부 확인
required = {"abs_time_ms", "run_id", "FB_Rpm", "FB_Torque", "Vib_rms"}
missing = required - set([c.strip() for c in df.columns])
missing

## 2) Day5 산출물 생성 (스크립트 실행)
- robust stats: median/MAD 기반
- score: `|robust z|` (샘플 단위) → (1s max)로 집계해 초 단위 이상치 표시
- 이벤트: 이상치 초들을 gap 기준으로 묶고(예: 2초 이내면 같은 이벤트), 각 이벤트의 peak_sec 기준으로 컨텍스트(±window) 그림 저장


In [ ]:
import subprocess
import sys

def run(cmd: str):
    print(">", cmd)
    res = subprocess.run(cmd, shell=True, text=True, capture_output=True)
    print(res.stdout)
    if res.returncode != 0:
        print(res.stderr)
    return res.returncode

py = sys.executable
run(f"{py} src/day5_anomaly_detection.py")

## 3) 생성 결과 확인 (파일 리스트)


In [ ]:
TABLES_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)
EVENT_DIR.mkdir(parents=True, exist_ok=True)

tables = sorted(TABLES_DIR.glob("*.csv"))
figs = sorted(FIG_DIR.glob("*.png"))
events = sorted(EVENT_DIR.glob("*.png"))

[t.name for t in tables], [f.name for f in figs], (len(events), events[:5])

## 4) 주요 그래프 확인
- (A) `day5_vib_robust_score_panel.png`: Vib_rms 컨텍스트 + |z| 패널
- (B) `day5_multisensor_overview.png`: RPM/Torque/Vib + |z| (이상치 초 빨간 점)


In [ ]:
from IPython.display import Image, display

p1 = FIG_DIR / "day5_vib_robust_score_panel.png"
p2 = FIG_DIR / "day5_multisensor_overview.png"

print(p1.name)
display(Image(filename=str(p1)))

print(p2.name)
display(Image(filename=str(p2)))

## 5) 이벤트 컨텍스트 확인 

In [ ]:
from IPython.display import Image, display

events_to_show = [
    "event_01_sec5852.png",
    "event_02_sec6315.png",
    "event_03_sec4870.png",
    "event_04_sec2438.png",
    "event_05_sec2208.png",
]

for name in events_to_show:
    p = EVENT_DIR / name
    print(p.name, "exists=", p.exists())
    if p.exists():
        display(Image(filename=str(p)))

## 6) 해석 요약
- Day5는 Vib_rms를 robust z-score로 점수화하고, 초 단위로 최대 |z|를 집계하여 이상치 후보 초를 표시한다.
- `day5_multisensor_overview.png`에서 이상치 초(빨간 점)가 나타나는 위치를 RPM/Torque/Vib의 맥락과 함께 확인할 수 있다.
- 이벤트 컨텍스트 그림은 peak_sec 기준 ±window에서 센서들이 어떻게 움직였는지를 한 번에 보여주며,
  요청한 5개 이벤트(`sec5852`, `sec6315`, `sec4870`, `sec2438`, `sec2208`)만 노트북에서 확인하도록 하였다.
